In [ ]:
from tvb.simulator.simulator import Simulator

import tvb.simulator.models
import tvb.simulator.integrators
import tvb.simulator.coupling
import tvb.simulator.monitors

from tvb.simulator.models.oscillator import Generic2dOscillator
from tvb.simulator.integrators import HeunStochastic
from tvb.simulator.coupling import Linear
from tvb.simulator.monitors import Raw, TemporalAverage
from tvb.datatypes.connectivity import Connectivity

import matplotlib.pyplot as plt
import numpy as np
import math
import pickle

In [ ]:
simulator_setup = {
    "connectivity": "Own",
    "model": "Generic2dOscillator",
    "integrator": "HeunStochastic",
    "coupling": "Difference",
    "monitor": "Raw",
}

In [ ]:
# Parameters
simulator_params = {
    "a_sim": 2.0,
    "b_sim": -10.0,
    "c_sim": 0.0,
    "d_sim": 0.02,
    "I_sim": 0.0,
    "nsig": 0.003,
    "dt": 1.0,
    "simulation_length": 1000
}

In [ ]:
connectivity = Connectivity()
connectivity.weights = np.array([[0., 2.], [2., 0.]])
connectivity.region_labels = np.array(["R1", "R2"])
connectivity.centres = np.array([[0.1, 0.1, 0.1], [0.2, 0.1, 0.1]])
connectivity.tract_lengths = np.array([[0., 2.5], [2.5, 0.]])
connectivity.configure()

In [ ]:
#tvb_data = '/Users/erichter/Documents/Uni/Charite/TVB_Distribution/tvb_data/tvb_data/'
#conn = Connectivity.from_file(tvb_data + 'connectivity/connectivity_76.zip')
#conn.configure()

In [ ]:
oscillator_model = getattr(tvb.simulator.models, simulator_setup["model"])(
    a=np.asarray([simulator_params["a_sim"]]),
    b=np.asarray([simulator_params["b_sim"]]),
    c=np.asarray([simulator_params["c_sim"]]),
    d=np.asarray([simulator_params["d_sim"]]),
    I=np.asarray([simulator_params["I_sim"]]),
)
oscillator_model.configure()

In [ ]:
integrator = getattr(tvb.simulator.integrators, simulator_setup["integrator"])(dt=simulator_params["dt"])
integrator.noise.nsig = np.array([simulator_params["nsig"]])
integrator.configure()

In [ ]:
coupling = getattr(tvb.simulator.coupling, simulator_setup["coupling"])()

In [ ]:
monitor = getattr(tvb.simulator.monitors, simulator_setup["monitor"])()

In [ ]:
sim = Simulator(
    model=oscillator_model,
    connectivity=connectivity,
    coupling=coupling,
    integrator=integrator,
    monitors=(monitor,),
    simulation_length=simulator_params["simulation_length"]
)

In [ ]:
sim.configure()

In [ ]:
(t, X), = sim.run()

In [ ]:
X.shape

In [ ]:
f1 = plt.figure(figsize=(14,8))
plt.plot(t, X[:, 0, 0, 0], label="R1")
plt.plot(t, X[:, 0, 1, 0], label="R2")
plt.xlabel("time (ms)")
plt.ylabel("states")
plt.legend()
plt.show()

In [ ]:
X[:, np.array([0]), :, :]

In [ ]:
with open("limit-cycle_simulation.pkl", "wb") as f:
    pickle.dump({**simulator_setup, **simulator_params, "simulation": X[:, np.array([0]), :, :], "x0": X[0]}, f)

In [ ]:
#np.save(f"limit-cycle_simulation", X)